## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Видим, что у большей части клиентов отрицательный рабочий стаж. Скорее всего, надо брать модуль. У некоторых пенсионеров стаж слишком большой, под 932 года, так не бывает, при том что даже вдесятеро меньший стаж тоже неправдоподобен. Есть пропуски данных в столбцах "Стаж" и "Ежемесячный доход", их одинаково в обоих столбцах, что позволяет предположить, что речь идет о безработных и без трудового стажа. Названия столбцов пригодны для дальнейшей обработки. По поводу стажа пенсионеров - если этот столбец будет важен для расчетов, то в этом случае нужно будет запрашивать уточнение у коллег. А пока попробуем его спасти другим способом.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
data['days_employed'].isna().sum()

2174

In [5]:
data['total_income'].isna().sum()

2174

В столбцах "Стаж" и "ежемесячный доход" равные количества NaN. меняем их на нули

In [6]:
data['days_employed'] = data['days_employed'].fillna(value=0)
data['total_income'] = data['total_income'].fillna(value=0)

Посмотрим на строки с нулями в столбцах "Стаж" и "Ежемесячный доход" - возможно, их лучше удалить

In [7]:
data[data['days_employed'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,0.0,сыграть свадьбу
26,0,0.0,41,среднее,1,женат / замужем,0,M,госслужащий,0,0.0,образование
29,0,0.0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,0.0,строительство жилой недвижимости
41,0,0.0,50,среднее,1,женат / замужем,0,F,госслужащий,0,0.0,сделка с подержанным автомобилем
55,0,0.0,54,среднее,1,гражданский брак,1,F,пенсионер,1,0.0,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,0.0,47,Среднее,1,женат / замужем,0,M,компаньон,0,0.0,сделка с автомобилем
21495,1,0.0,50,среднее,1,гражданский брак,1,F,сотрудник,0,0.0,свадьба
21497,0,0.0,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,0.0,строительство недвижимости
21502,1,0.0,42,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,строительство жилой недвижимости


Здесь есть госслужащие с нулевыми доходами и стажами, пенсионеры с нулевым доходом. Это люди разных возрастов, разных занятий, нет ничего, что бы их объединяло. Заполнять эти нули средними значениями - некорректно. Несмотря на то, что данные столбца "Стаж" в задаче не используются, они могут пригодиться в дальнейшем, поэтому и просто медианой заполнить эти нули - две с лишним тысячи строк - неправильно. Есть смысл заполнить эти нули медианами в зависимости от возраста. Выделим следующие возрастные группы: 0-18; 18-30; 30-50; 50 и далее лет.  

Посмотрим, сколько отрицательных значений в столбце "Стаж"

In [8]:
data[data['days_employed'] < 0]['debt'].count()

15906

Явно ошибка. На всякий случай сменим знак

In [9]:
data['days_employed'] = abs(data['days_employed'])
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [10]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [11]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [12]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'M'
data['gender'].unique()

array(['F', 'M'], dtype=object)

Теперь можно посмотреть на медианные доходы разных возрастных групп

In [13]:
data[data['dob_years'] <= 18]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,0.000000,0,среднее,1,женат / замужем,0,F,сотрудник,0,0.000000,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Как много младенцев с приличным доходом, стажем, с детьми и в разводе. А ведь это тоже пропуски. Посмотрим, сколько клиентов с нулевым возрастом.

In [14]:
data[data['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,0.000000,0,среднее,1,женат / замужем,0,F,сотрудник,0,0.000000,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Та же 101 строка. Значит, нужно начать с замены нулевых возрастов на медианные, скажем, по типу занятости.

In [15]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [16]:
sotr_avg = data[(data['income_type'] == 'сотрудник') & (data['dob_years'] != 0) ]['dob_years'].median()
sotr_avg

39.0

In [17]:
pens_avg = data[(data['income_type'] == 'пенсионер') & (data['dob_years'] != 0) ]['dob_years'].median()
pens_avg

60.0

In [18]:
comp_avg = data[(data['income_type'] == 'компаньон') & (data['dob_years'] != 0) ]['dob_years'].median()
comp_avg

39.0

In [19]:
gos_avg = data[(data['income_type'] == 'госслужащий') & (data['dob_years'] != 0) ]['dob_years'].median()
gos_avg

40.0

In [20]:
no_job_avg = data[(data['income_type'] == 'безработный') & (data['dob_years'] != 0) ]['dob_years'].median()
no_job_avg

38.0

In [21]:
business_avg = data[(data['income_type'] == 'предприниматель') & (data['dob_years'] != 0) ]['dob_years'].median()
business_avg

42.5

In [22]:
student_avg = data[(data['income_type'] == 'студент') & (data['dob_years'] != 0) ]['dob_years'].median()
student_avg

22.0

In [23]:
decret_avg = data[(data['income_type'] == 'в декрете') & (data['dob_years'] != 0) ]['dob_years'].median()
decret_avg

39.0

У категорий "сотрудник", "компаньон", "в декрете" равные медианы - 39 лет. Заполним пропуски.

In [24]:
def dob_age(row):
    if row['dob_years'] == 0:
        if row['income_type'] == 'сотрудник' or row['income_type'] == 'компаньон' or row['income_type'] == 'в декрете':
            row['dob_years'] = 39
        elif row['income_type'] == 'пенсионер':
            row['dob_years'] = 60
        elif row['income_type'] == 'госслужащий':
            row['dob_years'] = 40
        elif row['income_type'] == 'безработный':
            row['dob_years'] = 38
        elif row['income_type'] == 'предприниматель':
            row['dob_years'] = 42.5
        elif row['income_type'] == 'студент':
            row['dob_years'] = 22
    return row['dob_years']

In [25]:
data['age'] = data.apply(dob_age, axis = 1)
data.tail(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу,53
21506,1,1556.249906,33,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145541.993320,свадьба,33
21507,1,79.832064,32,среднее,1,гражданский брак,1,F,госслужащий,0,98180.279152,сделка с подержанным автомобилем,32
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость,62
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью,59
21510,2,0.000000,28,среднее,1,женат / замужем,0,F,сотрудник,0,0.000000,приобретение автомобиля,28
21511,0,612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи,29
21512,0,165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования,26
21513,0,1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья,35
21514,0,280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости,27


In [26]:
data.loc[99]

children                          0
days_employed                346542
dob_years                         0
education                   Среднее
education_id                      1
family_status       женат / замужем
family_status_id                  0
gender                            F
income_type               пенсионер
debt                              0
total_income                71291.5
purpose                  автомобиль
age                              60
Name: 99, dtype: object

В 99-й строке у нас был пенсионер без возраста. Теперь у него указан медианный для его типа занятости возраст. Значит, нам больше не нужен столбец "dob_years". Проверим, не осталось ли нулей в "age", и удалим лишний столбец.

In [27]:
data[data['age'] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age                 0
dtype: int64

In [28]:
data = data[['children', 'days_employed', 'age', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [29]:
data[data['age'] <= 18]

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


В таблице не осталось никого младше 18 лет. Это логично.

In [30]:
income_youth = data[(data['age'] > 18) & (data['age'] < 30) & (data['total_income'] != 0)]['total_income'].median()
income_youth

142141.58277574496

In [31]:
income_mid = data[(data['age'] >= 30) & (data['age'] < 50) & (data['total_income'] != 0)]['total_income'].median()
income_mid

154521.0953582153

In [32]:
income_old = data[(data['age'] >= 50) & (data['total_income'] != 0)]['total_income'].median()
income_old

133642.28073919698

Тоже логичные цифры: люди среднего возраста зарабатывают больше остальных, на втором месте молодые люди (от 18 до 30 лет), замыкают рейтинг пенсионеры. Этими данными можно заполнить пропуски в столбце "Ежемесячный доход".

In [33]:
def income(row):
    income_youth = 142141.58277574496
    income_mid = 154521.0953582153
    income_old = 133642.28073919698
    if row['total_income'] == 0:
        if row['age'] > 18 and row['age'] < 30:
            return income_youth
        elif row['age'] >= 30 and row['age'] < 50:
            return income_mid
        elif row['age'] >= 50:
            return income_old
    return row['total_income']            

In [34]:
data['income'] = data.apply(income, axis = 1)
data[data['total_income'] == 0].head(10)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income
12,0,0.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,0.0,сыграть свадьбу,133642.280739
26,0,0.0,41,среднее,1,женат / замужем,0,M,госслужащий,0,0.0,образование,154521.095358
29,0,0.0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,0.0,строительство жилой недвижимости,133642.280739
41,0,0.0,50,среднее,1,женат / замужем,0,F,госслужащий,0,0.0,сделка с подержанным автомобилем,133642.280739
55,0,0.0,54,среднее,1,гражданский брак,1,F,пенсионер,1,0.0,сыграть свадьбу,133642.280739
65,0,0.0,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,0.0,операции с коммерческой недвижимостью,142141.582776
67,0,0.0,52,высшее,0,женат / замужем,0,F,пенсионер,0,0.0,покупка жилья для семьи,133642.280739
72,1,0.0,32,высшее,0,женат / замужем,0,M,госслужащий,0,0.0,операции с коммерческой недвижимостью,154521.095358
82,2,0.0,50,высшее,0,женат / замужем,0,F,сотрудник,0,0.0,жилье,133642.280739
83,0,0.0,52,среднее,1,женат / замужем,0,M,сотрудник,0,0.0,жилье,133642.280739


In [35]:
data = data[['children', 'days_employed', 'age', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'income', 'purpose']]
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [36]:
data[data['days_employed'] == 0]

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose
12,0,0.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,133642.280739,сыграть свадьбу
26,0,0.0,41,среднее,1,женат / замужем,0,M,госслужащий,0,154521.095358,образование
29,0,0.0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,133642.280739,строительство жилой недвижимости
41,0,0.0,50,среднее,1,женат / замужем,0,F,госслужащий,0,133642.280739,сделка с подержанным автомобилем
55,0,0.0,54,среднее,1,гражданский брак,1,F,пенсионер,1,133642.280739,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,0.0,47,Среднее,1,женат / замужем,0,M,компаньон,0,154521.095358,сделка с автомобилем
21495,1,0.0,50,среднее,1,гражданский брак,1,F,сотрудник,0,133642.280739,свадьба
21497,0,0.0,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,154521.095358,строительство недвижимости
21502,1,0.0,42,среднее,1,женат / замужем,0,F,сотрудник,0,154521.095358,строительство жилой недвижимости


### Вывод

В этой части заменили пропущенные значения в столбце "возраст" медианными значениями по типу занятости, поскольку заполнять их просто средним или медианой нелогично - получим сорокалетних студентов и пенсионеров. В столбце "Ежемесячный доход" заменили пропущенные значения медианами по возрастным группам, поскольку логично, что разные возрастные группы зарабатывают по-разному. Также избавились от отрицательного стажа. Кроме того, нашли пропуск в столбце 'gender' - всего одна строка, по совокупности качеств можно предположить, что это мужчина, поэтому заменили XNA на М. Однако и этот момент тоже лучше уточнить у коллег - возможно, с их помощью удастся восстановить точную картину. 

### Замена типа данных

In [37]:
data['days_employed'] = data['days_employed'].astype('int')
data['income'] = data['income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
income              21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [38]:
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Видим, что перевод из float в int просто отбрасывает десятичную часть. в контексте данной задачи этим можно пренебречь, в других случаях лучше округлять. Так как нужно было перевести в нужный формат весь столбец, то выбран именно метод astype()

### Обработка дубликатов

In [39]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [40]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [41]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [42]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [43]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [44]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [45]:
data.duplicated().sum()

71

Нашлись строки, которые полностью дублируются аж 71 раз. Проверим

In [46]:
data[data.duplicated(keep = False)].head(20)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose
120,0,0,46,среднее,1,женат / замужем,0,F,сотрудник,0,154521,высшее образование
520,0,0,35,среднее,1,гражданский брак,1,F,сотрудник,0,154521,сыграть свадьбу
541,0,0,57,среднее,1,женат / замужем,0,F,сотрудник,0,133642,сделка с подержанным автомобилем
554,0,0,60,среднее,1,женат / замужем,0,M,сотрудник,0,133642,покупка недвижимости
680,1,0,30,высшее,0,женат / замужем,0,F,госслужащий,0,154521,покупка жилья для семьи
1005,0,0,62,среднее,1,женат / замужем,0,F,пенсионер,0,133642,ремонт жилью
1191,0,0,61,среднее,1,женат / замужем,0,F,пенсионер,0,133642,операции с недвижимостью
1431,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,154521,покупка жилья для семьи
1511,0,0,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,133642,дополнительное образование
1681,0,0,57,среднее,1,гражданский брак,1,F,пенсионер,0,133642,на проведение свадьбы


У всех нулевой стаж, что характерно. Я хотел бы убедиться в том, что такой поиск дубликатов действительно работает. Проверим дубликаты для строки 120.

In [47]:
data[(data['age'] == 46) & (data['days_employed'] == 0) & (data['children'] == 0) & (data['family_status_id'] == 0) & (data['education_id'] == 1)]

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose
120,0,0,46,среднее,1,женат / замужем,0,F,сотрудник,0,154521,высшее образование
2743,0,0,46,среднее,1,женат / замужем,0,F,сотрудник,0,154521,покупка жилья для сдачи
3533,0,0,46,среднее,1,женат / замужем,0,M,сотрудник,0,154521,жилье
5648,0,0,46,среднее,1,женат / замужем,0,M,госслужащий,1,154521,приобретение автомобиля
7623,0,0,46,среднее,1,женат / замужем,0,M,пенсионер,1,154521,покупка жилья
9009,0,0,46,среднее,1,женат / замужем,0,F,сотрудник,0,154521,свой автомобиль
9982,0,0,46,среднее,1,женат / замужем,0,F,сотрудник,0,154521,строительство жилой недвижимости
10765,0,0,46,среднее,1,женат / замужем,0,F,сотрудник,1,154521,заняться образованием
13233,0,0,46,среднее,1,женат / замужем,0,F,компаньон,0,154521,покупка своего жилья
16378,0,0,46,среднее,1,женат / замужем,0,F,сотрудник,0,154521,высшее образование


Да, у строки 120 есть дубль в строке 16378. Эти дубликаты надо удалять.

In [48]:
data = data.drop_duplicates().reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
age                 21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
income              21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Было 21525 строк, стало 21454 строки - ровно на 71 строку стало меньше. Значит, все верно и удаление дубликатов сработало как нужно.

### Вывод

В столбце 'education' нашлись дубликаты по регистру, от таких дубликатов проще и надежнее всего избавляться приведением к нижнему регистру. Причины появления таких дубликатов кроются в ручном вводе вариантов уровня образования. Если бы вместо этого был выпадающий список из строго определенного числа вариантов, то этой ошибки бы не возникло. Также на всякий случай привели к единому виду (в нижний регистр) набор вариантов из столбца family_status, хотя это необязательно. Кроме того, удалили дубликаты полных строк. А вот дубликаты в столбце 'purpose' требуют лемматизации; более того, нужно будет разобраться с синонимами, поскольку жилье и жилая недвижимость - это одно и то же.

### Лемматизация

In [49]:
def lemmnik(i):
    lemmas = ' '.join(m.lemmatize(i))
    return lemmas

In [50]:
data['lemmas_purpose'] = data['purpose'].apply(lemmnik)
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose,lemmas_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n


In [51]:
def categorize(i):
    if 'жил' in i or (('собственный' in i) and 'недвижимость' in i):
        return 'жилье'
    elif 'недвижимость' in i:
        return 'коммерческая недвижимость'
    elif 'автомобиль' in i:
        return 'автомобиль'
    elif 'образование' in i:
        return 'образование'
    elif 'свадьба' in i:
        return 'свадьба'

In [52]:
data['purpose_category'] = data['lemmas_purpose'].apply(categorize)
data.head(10)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose,lemmas_purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n,жилье


In [53]:
data[data['purpose_category'] == 'жилье'].head(20)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose,lemmas_purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,жилье
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,жилье
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n,жилье
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n,жилье
14,0,1844,56,высшее,0,гражданский брак,1,F,компаньон,1,165127,покупка жилой недвижимости,покупка жилой недвижимость \n,жилье
15,1,972,26,среднее,1,женат / замужем,0,F,сотрудник,0,116820,строительство собственной недвижимости,строительство собственный недвижимость \n,жилье
27,0,529,28,высшее,0,женат / замужем,0,M,сотрудник,0,308848,строительство собственной недвижимости,строительство собственный недвижимость \n,жилье
28,1,717,26,высшее,0,женат / замужем,0,F,сотрудник,0,187863,строительство собственной недвижимости,строительство собственный недвижимость \n,жилье
29,0,0,63,среднее,1,не женат / не замужем,4,F,пенсионер,0,133642,строительство жилой недвижимости,строительство жилой недвижимость \n,жилье


In [54]:
data = data[['children', 'days_employed', 'age', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'income', 'purpose_category']]
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


### Вывод

Все варианты целей получения кредита можно условно разбить на пять основных категорий: жилье, автомобиль, образование, свадьба, коммерческая недвижимость. Чтобы лемматизировать столбец 'purpose', написали функцию, которая получает набор лемм, разделенных пробелом, в виде строки, и добавили служебный столбец с леммами к исходной таблице. Затем написали еще одну функцию, которая ищет в списке лемм ключевое слово и по нему возвращает нужную категорию; также добавили столбец с полученными категориями к исходной таблице, а затем удалили столбец с леммами и исходный столбец 'purpose', чтобы не засорять таблицу. Теперь в целях кредита наведен порядок, и данные можно категоризировать.

### Категоризация данных

In [55]:
def categories(i):
    if i == 'жилье':
        return 0
    elif i == 'автомобиль':
        return 1
    elif i == 'образование':
        return 2
    elif i == 'свадьба':
        return 3
    elif i == 'коммерческая недвижимость':
        return 4

    

In [56]:
data.loc[:, 'categories_id'] = data['purpose_category'].apply(categories)

In [57]:
data.head(10)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose_category,categories_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,2
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,3
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,жилье,0
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,жилье,0
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,2
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,3
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,жилье,0


In [58]:
def children_group(i):
    if i == 0:
        return 0
    else:
        return 1    

In [59]:
data['children_id'] = data['children'].apply(children_group)

In [60]:
data.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,income,purpose_category,categories_id,children_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье,0,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,1,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье,0,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,2,1
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,3,0


### Вывод

Для категоризации по цели кредита был взят следующий словарь: жилье - 0, автомобиль - 1, образование - 2, свадьба - 3,
коммерческая недвижимость - 4

Также добавлена категоризация по наличию детей: в столбце children_id стоит 0, если у клиента нет детей, и 1, если есть.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [61]:
parents = data[data['children_id'] == 1]['children_id'].count()
parents

7363

In [62]:
childless = data[data['children_id'] == 0]['children_id'].count()
childless

14091

Бездетных вдвое больше, чем родителей.

In [63]:
parents_debt = data[(data['children_id'] == 1) & (data['debt'] == 1)]['children'].count()
parents_debt

678

In [64]:
childless_debt = data[(data['children_id'] == 0) & (data['debt'] == 1)]['children'].count()
childless_debt

1063

In [65]:
part_of_debtors_in_parents = parents_debt / parents 
part_of_debtors_in_childless = childless_debt / childless 

In [66]:
print('Процент должников среди родителей: {:.2%}'.format(part_of_debtors_in_parents))

Процент должников среди родителей: 9.21%


In [67]:
print('Процент должников среди бездетных: {:.2%}'.format(part_of_debtors_in_childless))

Процент должников среди бездетных: 7.54%


In [68]:
print('В случае рождения детей вероятность стать должником по кредиту возрастает на {:.2%}'.format(1 - part_of_debtors_in_childless / part_of_debtors_in_parents))

В случае рождения детей вероятность стать должником по кредиту возрастает на 18.07%


### Вывод

Видим, что бездетных заемщиков вдвое больше, чем заемщиков-родителей, поэтому прямое сравнение не работает. Процент должников среди бездетных заемщиков почти на 20% меньше, чем среди заемщиков-родителей. Таким образом, в случае рождения детей вероятность родителей стать должниками по кредиту вырастает на 20%. Тем не менее, в данной выборке должниками становятся не более 10% клиентов независимо от наличия детей в их семьях.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [69]:
famstatus_dict = data[['family_status_id', 'family_status']]
famstatus_dict.head()

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак


In [70]:
famstatus_dict = famstatus_dict.drop_duplicates().reset_index(drop = True)
famstatus_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [71]:
family_pivot = data.pivot_table(index = 'family_status', aggfunc={'family_status_id':len, 'debt':sum})
family_pivot

,debt,family_status_id
family_status,,
в разводе,85,1195
вдовец / вдова,63,959
гражданский брак,388,4151
женат / замужем,931,12339
не женат / не замужем,274,2810


In [72]:
family_pivot['ratio'] = family_pivot['debt'] / family_pivot['family_status_id'] * 100
family_pivot['ratio'] = family_pivot['ratio'].astype(int)
family_pivot.sort_values(by = 'ratio', ascending = False)

,debt,family_status_id,ratio
family_status,,,
гражданский брак,388,4151,9
не женат / не замужем,274,2810,9
в разводе,85,1195,7
женат / замужем,931,12339,7
вдовец / вдова,63,959,6


### Вывод

В процентном соотношении должниками чаще становятся неженатые/незамужние клиенты - в среднем каждый десятый. При этом семейные, несмотря на то что с отрывом чаще всех остальных категорий берут кредиты, должниками становятся реже. Реже всех берут кредиты и реже всех становятся должниками вдовствующие клиенты.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [73]:
total_income = pd.qcut(data['income'], 4)
income_pivot = data.pivot_table(index=[total_income], aggfunc={'family_status_id':len, 'debt':sum})
income_pivot

,debt,family_status_id
income,,
"(20666.999, 107623.0]",427,5364
"(107623.0, 145228.0]",458,5363
"(145228.0, 195813.25]",473,5363
"(195813.25, 2265604.0]",383,5364


In [74]:
income_pivot['part'] = income_pivot['debt'] / income_pivot['family_status_id'] * 100
#income_pivot['part'] = income_pivot['part'].astype(int)
income_pivot.sort_values(by='part', ascending = False)

,debt,family_status_id,part
income,,,
"(145228.0, 195813.25]",473,5363,8.819690
"(107623.0, 145228.0]",458,5363,8.539996
"(20666.999, 107623.0]",427,5364,7.960477
"(195813.25, 2265604.0]",383,5364,7.140194


### Вывод

Если разбить квантилями столбец "Уровень дохода", получим четыре примерно равных по численности группы с медианным доходом в 145000 руб/мес. Наибольший процент должников - в группе чуть выше медианы, с доходом от медианы до примерно 200 000 руб/мес, а клиенты с наименьшими и наибольшими доходами реже остальных задерживают платежи по кредитам. Однако разница столь мала, что ей можно пренебречь: между наибольшей и наименьшей долей должников - 1,8%. Таким образом, уровень дохода практически не влияет на возврат кредита в срок. Можно только сказать, что клиенты с наибольшими и наименьшими заработками, скорее всего, будут аккуратнее относиться к своим обязательствам.

- Как разные цели кредита влияют на его возврат в срок?

In [75]:
purpose_pivot = data.pivot_table(index = 'purpose_category', aggfunc={'categories_id':len, 'debt':sum})
purpose_pivot

,categories_id,debt
purpose_category,,
автомобиль,4306,403
жилье,6325,439
коммерческая недвижимость,4486,343
образование,4013,370
свадьба,2324,186


In [76]:
purpose_pivot['part'] = purpose_pivot['debt'] / purpose_pivot['categories_id'] * 100
purpose_pivot.sort_values(by='part', ascending = False)

,categories_id,debt,part
purpose_category,,,
автомобиль,4306,403,9.359034
образование,4013,370,9.220035
свадьба,2324,186,8.003442
коммерческая недвижимость,4486,343,7.646010
жилье,6325,439,6.940711


### Вывод

Наибольший процент должников - среди тех, кто берет кредит на автомобиль или образование, наименьший - среди тех, кто берет кредит на операции с жильем. Разброс составляет примерно 2,5%, и его вряд ли можно списать на погрешность. Поэтому резюмируем, что зависимость между целью кредита и возвратом его в срок существует.

### Шаг 4. Общий вывод

В исходной таблице обнаружилось много строк с нулями (NaN) в столбцах "Стаж" и "Доход". Заполнение этих строк медианными значениями представляется некорректным, поскольку могло исказить общую картину. Эти строки пришлось удалить, оставив в выборке 19 351 строку. Проанализировав данные, видим следующее: 
1. процент должников среди родителей заметно выше, чем среди бездетных заемщиков: в случае рождения детей вероятность стать должником по кредиту возрастает на 18.07%; 
2. Также есть явная зависимость между семейным положением и возвратом кредита в срок: неженатые / незамужние клиенты чаще всего становятся должниками (в среднем кадый десятый). При этом можно смело объединить категории неженатых / незамужних и живущих в гражданском браке: отсутствие пресловутого "штампа в паспорте" существенно увеличивает вероятность просрочки платежей. А вот семейные клиенты и вдовцы - самые аккуратные плательщики.
3. Между уровнем дохода и возвратом кредита в срок зависимость минимальна: между наибольшей и наименьшей долей должников - 1,8%.
4. Цель кредита влияет на вероятность просрочки платежей: если цель кредита - автомобиль или образование, то вероятность явно выше, чем если целью кредита являются операции с жильем. Разброс составляет примерно 2,5%.